In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder\
        .appName("spark practice 13mar2025")\
        .getOrCreate()

print(spark)

25/03/17 09:53:16 WARN Utils: Your hostname, TTNPL-8203 resolves to a loopback address: 127.0.1.1; using 10.1.243.232 instead (on interface wlp0s20f3)
25/03/17 09:53:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/17 09:53:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.sql.types import *

In [5]:
data = [(1,"abc",["java","scala"]),
        (2,"xyz",["python","R"]),
        (3,"mno",["scala","javaScript"])
       ]

schema = StructType([
    StructField("Id",IntegerType(),True),
    StructField("name",StringType(),True),
    StructField("languages",ArrayType(StringType()) , True)
])


df = spark.createDataFrame(data =data , schema =schema)

In [6]:
df.show()

+---+----+-------------------+
| Id|name|          languages|
+---+----+-------------------+
|  1| abc|      [java, scala]|
|  2| xyz|        [python, R]|
|  3| mno|[scala, javaScript]|
+---+----+-------------------+



#### using isin(_)

In [7]:
from pyspark.sql.functions import explode

In [8]:
listvalues = ["java","scala"]

In [9]:
df2 = df.withColumn("languages",explode(df.languages))


In [10]:
print("df2 is ")
df2.show()

df2.filter(df2.languages.isin(listvalues)).show()

df2 is 
+---+----+----------+
| Id|name| languages|
+---+----+----------+
|  1| abc|      java|
|  1| abc|     scala|
|  2| xyz|    python|
|  2| xyz|         R|
|  3| mno|     scala|
|  3| mno|javaScript|
+---+----+----------+

+---+----+---------+
| Id|name|languages|
+---+----+---------+
|  1| abc|     java|
|  1| abc|    scala|
|  3| mno|    scala|
+---+----+---------+



In [11]:
df2.filter(~ df2.languages.isin(listvalues)).show()

+---+----+----------+
| Id|name| languages|
+---+----+----------+
|  2| xyz|    python|
|  2| xyz|         R|
|  3| mno|javaScript|
+---+----+----------+



In [12]:
df2.filter("languages NOT IN ('java','scala')").show()

+---+----+----------+
| Id|name| languages|
+---+----+----------+
|  2| xyz|    python|
|  2| xyz|         R|
|  3| mno|javaScript|
+---+----+----------+



#### Group By functionality

In [13]:
data_e = [
    (1,"abc",1,35000.0),
    (2,"xyz",2,45000.0),
    (3,"mno",3,42000.0),
    (4,"klm",1,34000.0),
    (5,"fgh",None,45000.0),
]

data_d = [
    (1,"IT"),
    (2,"HR"),
    (3,"Finance"),
    (4,"Unofficial")
]

schema_e = StructType([
    StructField("id",IntegerType(),True),
    StructField("name",StringType(),True),
    StructField("Dept",IntegerType(),True),
    StructField("salary",FloatType(),True)
])

schema_d = StructType([
    StructField("dept_code",IntegerType(),True),
    StructField("dept",StringType(),True)
])

emp_df = spark.createDataFrame(data = data_e,schema = schema_e)
dept_df = spark.createDataFrame(data =data_d,schema =schema_d)

In [14]:
emp_df.show()
dept_df.show()

+---+----+----+-------+
| id|name|Dept| salary|
+---+----+----+-------+
|  1| abc|   1|35000.0|
|  2| xyz|   2|45000.0|
|  3| mno|   3|42000.0|
|  4| klm|   1|34000.0|
|  5| fgh|NULL|45000.0|
+---+----+----+-------+

+---------+----------+
|dept_code|      dept|
+---------+----------+
|        1|        IT|
|        2|        HR|
|        3|   Finance|
|        4|Unofficial|
+---------+----------+



#### joins in the spark

In [15]:
emp_df.join(dept_df,emp_df.Dept==dept_df.dept_code,"outer").show()

[Stage 22:>                                                         (0 + 8) / 8]

+----+----+----+-------+---------+----------+
|  id|name|Dept| salary|dept_code|      dept|
+----+----+----+-------+---------+----------+
|   5| fgh|NULL|45000.0|     NULL|      NULL|
|   4| klm|   1|34000.0|        1|        IT|
|   1| abc|   1|35000.0|        1|        IT|
|   2| xyz|   2|45000.0|        2|        HR|
|   3| mno|   3|42000.0|        3|   Finance|
|NULL|NULL|NULL|   NULL|        4|Unofficial|
+----+----+----+-------+---------+----------+



In [16]:
df3 = emp_df.join(dept_df,emp_df.Dept==dept_df.dept_code,"semi")


In [17]:
emp_df.join(dept_df,emp_df.Dept==dept_df.dept_code,"semi").select("*").orderBy(emp_df.id).show()


+---+----+----+-------+
| id|name|Dept| salary|
+---+----+----+-------+
|  1| abc|   1|35000.0|
|  2| xyz|   2|45000.0|
|  3| mno|   3|42000.0|
|  4| klm|   1|34000.0|
+---+----+----+-------+



In [18]:
emp_df.join(dept_df,emp_df.Dept==dept_df.dept_code,"anti").select("*").orderBy(emp_df.id).show()

+---+----+----+-------+
| id|name|Dept| salary|
+---+----+----+-------+
|  5| fgh|NULL|45000.0|
+---+----+----+-------+



In [19]:
emp_df.join(dept_df,emp_df.Dept==dept_df.dept_code,"semi").select("*").orderBy(emp_df.id).show()


+---+----+----+-------+
| id|name|Dept| salary|
+---+----+----+-------+
|  1| abc|   1|35000.0|
|  2| xyz|   2|45000.0|
|  3| mno|   3|42000.0|
|  4| klm|   1|34000.0|
+---+----+----+-------+



In [20]:
emp_df.join(dept_df,emp_df.Dept==dept_df.dept_code,"semi").select("*").select(emp_df.id,)

DataFrame[id: int]

In [21]:
emp_df.show()

+---+----+----+-------+
| id|name|Dept| salary|
+---+----+----+-------+
|  1| abc|   1|35000.0|
|  2| xyz|   2|45000.0|
|  3| mno|   3|42000.0|
|  4| klm|   1|34000.0|
|  5| fgh|NULL|45000.0|
+---+----+----+-------+



In [22]:
spark.stop()